In [16]:
GPUS = [2]
PATH = "final_data/data"

HEIGHT = 160
WIDTH = 160
BATCH_SIZE = 16

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, GPUS))

In [17]:
import cv2
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import metrics
from adversarial.utils import dataloader
from adversarial import models

In [18]:
target_model = models.TargetModelV3()
target_model(tf.random.normal((10, 160, 160, 3)), tf.random.normal((10,)), True)
target_model.load_custom_weights("model/ensemble_distinction-2020-06-05_14-28-15.pkl")

In [19]:
original_model = models.MobileNetV2()
original_model(tf.random.normal((10, 160, 160, 3)))
original_model.load_custom_weights("data/model_20200507_9_1.00_0.0088")

In [20]:
acc = metrics.SparseCategoricalAccuracy()
test_dloader = dataloader.DataLoader("data", BATCH_SIZE, training=False)

In [21]:
# plt.imread("final_data/affend_1/1_327_0png.png")[..., -1]

In [22]:
def test_data_load():
    data_list = []
    
    for f in pathlib.Path(PATH).glob("*.png"):
        image = plt.imread(str(f), format="png")
        
        if "_" in f.name:
            index = int(f.name.split("_")[1])
            label = int(f.name.split("_")[2][0])
        elif "-" in f.name:
            index = int(f.name.split("-")[1])
            label = int(f.name.split("-")[2][0])
        
        if image.shape[-1] == 4:
            image = image[..., :3]
            
        if image.shape[0] != 160 or image.shape[1] != 160:
            image = cv2.resize(image, dsize=(WIDTH, HEIGHT))
        
        data_list.append((image, label, index))
        
    return data_list

In [23]:
data_list = test_data_load()
len(data_list)

100

In [24]:
x_test = tf.stack(list(map(lambda tup: tup[0], data_list)), axis=0)
y_test = tf.stack(list(map(lambda tup: tup[1], data_list)), axis=0)

print(tf.shape(x_test))
print(tf.shape(y_test))

tf.Tensor([100 160 160   3], shape=(4,), dtype=int32)
tf.Tensor([100], shape=(1,), dtype=int32)


In [25]:
def test_clean():
    clean_images = []
    
    for x, y, i in data_list:
        clean_images.append(test_dloader[i][0])
        
    clean_images = tf.stack(clean_images, axis=0)
    
    pred = original_model(clean_images, training=False)
    acc(y_test, pred)
    
    test_acc = acc.result()
    acc.reset_states()
    
    return test_acc

In [26]:
def test_adv_with_original_model():
    
    pred = original_model(x_test, training=False)
    acc(y_test, pred)
    
    test_acc = acc.result()
    acc.reset_states()
    
    return test_acc

In [27]:
def test_adv_with_target_model():
    
    pred = target_model(x_test, labels=None, training=False)
    acc(y_test, pred)
    
    test_acc = acc.result()
    acc.reset_states()
    
    return test_acc

In [28]:
test_acc = test_clean()
print(test_acc)

tf.Tensor(0.94, shape=(), dtype=float32)


In [29]:
test_acc = test_adv_with_original_model()
print(test_acc)

tf.Tensor(0.85, shape=(), dtype=float32)


In [30]:
test_acc = test_adv_with_target_model()
print(test_acc)

tf.Tensor(0.93, shape=(), dtype=float32)
